![Wolfe](https://s3.amazonaws.com/lquant-images/wolfe_luo.jpg)

## Overview of Black Litterman API

Features:
1. Upload custom data 
2. Use factors/price target level view 




### Data for Black Litterman

A simple pandas data frame with DATE/TICKER/SEDOL can be used for runing black litterman. The columns in CSV file can be referenced in the API to set factor scores, price target and recommendation associate with the stock

In [15]:
import pandas as pd
df = pd.read_csv('black-litterman-test-data.csv')
df.head()

,DATE,MALTA,BM_QUANT,BALI,GRESSA,TALIA,MALESSA,DEPTH,CARE,TICKER,PRIOR_RETURN,PRCCD,TP,ES_RECOMM_AVG,RECOMMENDATION
0,2020-12-31,-1.170451,-3.544589,NaN,NaN,NaN,NaN,NaN,NaN,AAL,0.059128,15.77,11.36111,4.0,strong buy
1,2020-12-31,0.333774,0.698127,NaN,NaN,NaN,NaN,NaN,NaN,PNW,0.024085,79.95,88.76923,3.0,buy
2,2020-12-31,-0.425797,0.768676,NaN,NaN,NaN,NaN,NaN,-0.146916,ABT,0.020055,109.49,120.47059,2.0,hold
3,2020-12-31,1.118058,0.113447,NaN,NaN,1.525873,NaN,NaN,NaN,AMD,0.029687,91.71,91.58750,2.0,hold
4,2020-12-31,0.155671,0.425559,NaN,NaN,NaN,NaN,NaN,NaN,APD,0.024811,273.22,307.35000,2.0,hold


## Requirements and Presteps

1. Copy pyqes [python file]( https://github.com/wolferesearch/docs/tree/master/micro-services/api/python/pyqes) from github to your local directory from Github. 
2. Ensure you have [Pandas](https://pandas.pydata.org/) and [requests](https://pypi.org/project/requests/) package in your python kernel. 

## Authentication and Connection

The API is protected using Username and Password. In case you have not received it, please [email](mailto:luo.qes@wolferesearch.com) to apply for API account. 

The connection object is the gateway to accessing the API. It allows you to access the catalog, portfolios, templates, risk models etc. 

In [1]:
## Open connection using username/password. 
from pyqes import micsvc
connection = micsvc.Connection(username='****', password = '*******')

In [16]:
connection.get_jobs()

,ENDTIME,MESSAGE,STARTTIME,STATUS,TYPEID,USER_,UUID
1,1675270141366,Job Completed,2023-02-01 16:26:17.492,SUCCESS,8,hjain,815ac724-8c39-4f04-af2b-4987176e8d82
2,1675268541268,"Error in doTryCatch(return(expr), name, parent...",2023-02-01 16:22:21.143,ERROR,1,hjain,2eff9a60-8fb0-418d-b6c3-9f775e4f531a
4,1675268426857,"Error in doTryCatch(return(expr), name, parent...",2023-02-01 16:20:26.647,ERROR,1,hjain,160050e0-2d10-4dbd-856d-ea5075e06a02
3,1675110043467,Job Completed 0,2023-01-30 20:20:26.707,SUCCESS,7,hjain,10751a63-81b9-4900-adee-a01e771273fb
0,1675107215947,Job Completed 0,2023-01-30 19:33:09.026,SUCCESS,7,hjain,f2bd799a-86ea-4070-a769-569dbaecba7c


#### Black Litterman Request Object

The black litterman object can be initiated using the connection class. Note that this object can be linked to a previously run black litterman request (using uuid) or you can run a new black litterman request. 

In [17]:
# Use connection object to get a new instance of black litterman
bl = connection.get_black_litterman()

#### Add Prior return

Note that string "PRIOR_RETURN" will be referenced in the pandas data frame

In [ ]:
## Set Prior Return
bl.set_prior_return(prior_return='PRIOR_RETURN', horizon = 21)

#### Add Factor View 1 (MALTA)

TODO: Add details of each parameter

In [ ]:
## Set Factor View
bl.add_view_model_scores(view_name = 'MALTA', model_scores = 'MALTA', return_horizon = 21, 
                         information_coefficient = 0.1,
                         uncertainty_kappa_empirical = {
                             'half_life': 126
                         })

#### Add Factor View 2 (BM_QUANT)



In [ ]:
bl.add_view_model_scores(view_name = 'BM_QUANT', model_scores = 'BM_QUANT', return_horizon = 21, 
                         information_coefficient = 0.05,
                         uncertainty_diagonal_empirical = {
                             'half_life': 126
                         })

#### Set data associated with the Black Litterman Optimization

We will bascially upload the pandas data frame that optimization will use

In [ ]:
bl.set_user_data(data = df, name = 'black-litterman-test-data.csv', overwrite = True)

#### Inspect the JSON object sent to server


#### Submit the Request to the Server

The request is submitted in async fashion. Caller can either periodically check the status of the request or wait for it using the bl.wait call. 

In [ ]:
bl.submit()

#### Check the Status of the Request

Ensure that a success status is received. If not please share the error with the luo.qes@wolferesearch.com 

In [ ]:
bl.info()

#### Getting the Ouput

Output is a nested data set that can be pulled from the server for the current request. It uses the UUID stored in the object to retrieve the data 



In [ ]:
output = bl.esvc.get_job_output()
output.files.head()

#### Buffer all data to the notebook. 

The below `get_data` call will pull all data to notebook as Python Dictionary object. 

In [ ]:
data = output.get_data()

#### Inspecting the Output

In [ ]:
data['result'].keys()

In [ ]:
data['result']['universe'].head()

In [ ]:
data['result']['uncertainties']

### Previously Run Jobs

In [ ]:
connection.get_jobs()

In [18]:
bl = connection.get_black_litterman()
bl.set_id('f2bd799a-86ea-4070-a769-569dbaecba7c')

In [19]:
print(bl.get_logs())

2023-01-30 19:33:09 - INFO - prior return horizon of 21 trading days
2023-01-30 19:33:09 - INFO - Setting View [MALTA] of Type [factor]
INFO [2023-01-30 19:33:09] view MALTA return horizon of 21 trading days
2023-01-30 19:33:09 - INFO - Setting View [BM_QUANT] of Type [factor]
INFO [2023-01-30 19:33:09] view BM_QUANT return horizon of 21 trading days
2023-01-30 19:33:09 - INFO - Start
2023-01-30 19:33:09 - INFO - Getting universe and mapping identifiers.
[1] "Coverage percentage:"
    BM_QUANT       MALTA        universe  
 Min.   :100   Min.   :100   Min.   :100  
 1st Qu.:100   1st Qu.:100   1st Qu.:100  
 Median :100   Median :100   Median :100  
 Mean   :100   Mean   :100   Mean   :100  
 3rd Qu.:100   3rd Qu.:100   3rd Qu.:100  
 Max.   :100   Max.   :100   Max.   :100  
[1] "ID types percentage"
gvkey_iid 
      100 
2023-01-30 19:33:12 - INFO - Loading covariance matrix
2023-01-30 19:33:13 - WARN - File type not recognized.
2023-01-30 19:33:13 - INFO - Getting risk model data fr

In [20]:
output = bl.get_output()

In [21]:
output.files

,Key,LastModified,Size
0,result/D_id_map.csv,2023-01-30 19:33:36+00:00,22487
1,result/ML_universe.csv,2023-01-30 19:33:36+00:00,38020
2,result/MN_coverage_percent.csv,2023-01-30 19:33:36+00:00,949
3,result/MN_posterior_return.csv,2023-01-30 19:33:36+00:00,123652
4,result/MN_prior_return.csv,2023-01-30 19:33:36+00:00,122294
5,result/SN_tau.csv,2023-01-30 19:33:36+00:00,13
6,result/SS_covariance_matrix.csv,2023-01-30 19:33:36+00:00,25
7,result/VN_return_horizon_days.csv,2023-01-30 19:33:36+00:00,95
8,result/posterior_covariance_matrix/2020-12-31/...,2023-01-30 19:33:36+00:00,214378
9,result/posterior_covariance_matrix/2020-12-31/...,2023-01-30 19:33:36+00:00,547128


In [ ]:
key = 'D_summary.csv'

In [ ]:
key[(key.find('_')+1):len(key)]

In [22]:
v = output.get_data(prefix = 'result/uncertainties/')

In [30]:
v['result']['uncertainties']['diagonal_uncertainties']['BM_QUANT']

,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30
K9RQE6VQGZ,0.183264,0.174982,0.174658,0.165753,0.168339,0.166941,0.094149,0.085702,0.080757,0.078870,0.080116,0.077612
04M5J6K6G4,0.088846,0.089326,0.085258,0.085678,0.083732,0.081171,0.080137,0.075003,0.074218,0.075050,0.076905,0.074417
04M5JN8NG4,0.070662,0.071606,0.068449,0.071210,0.070584,0.069498,0.069532,0.067301,0.067198,0.068227,0.071259,0.069246
04M5O1EOK4,0.109873,0.110853,0.110352,0.123407,0.126498,0.124873,0.077596,0.075465,0.071264,0.075720,0.079912,0.096011
04M5YO72K4,0.085336,0.086773,0.082245,0.083562,0.081768,0.080593,0.078543,0.073356,0.071293,0.070945,0.080519,0.077805
04M5752JK4,0.162941,0.161762,0.157008,0.158941,0.154557,0.150746,0.093448,0.085217,0.080033,0.078138,0.081680,0.077960
04M5KM6KG4,0.093371,0.093323,0.088436,0.090615,0.088416,0.086462,0.070602,0.067014,0.063596,0.066265,0.068535,0.066405
04M5K1Q6G4,0.114481,0.116887,0.119457,0.133645,0.135998,0.134613,0.077067,0.072748,0.068502,0.069637,0.070828,0.069954
04M5PLVYK4,0.165775,0.164966,0.163701,0.169404,0.168595,0.165145,0.080245,0.093749,0.091064,0.091450,0.087400,0.087225
04M5863MK4,0.076871,0.076938,0.074963,0.079015,0.077279,0.075033,0.070196,0.067798,0.064462,0.067602,0.068291,0.066054
